In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import beta
import scipy
from sklearn import preprocessing
from datetime import datetime
from datetime import date, time, datetime
import os, pandas as pd, glob
import pandas.io.common

In [3]:
path = "/Users/Jeanne/Documents/personal/Auction Results/OBS/Excel"
files_list = glob.glob(os.path.join(path, "*.xls"))
files_listx = glob.glob(os.path.join(path, "*.xlsx"))
for f in files_listx:
    files_list.append(f)

headers = ('Hip #','Sex','Sire','Dam','Consignor','Buyer','Price','PS')
#df = pd.DataFrame(columns=headers)
df = pd.DataFrame()
for f in files_list:
    temp = pd.read_excel(f,nrows = 20)
    row = temp.iloc[:,7].first_valid_index()
    
    temp = pd.read_excel(f,header = row+1)
    y = 2000 + int(f[64:66])
    m = int(f[62:64])
    temp['OBSDate'] = date(year=y, month=m, day=1)
    df = df.append(temp)
    
df.head()

,hip#,NAME,sex,year,SIRE,DAM,IN FOAL SIRE,CONSIGNOR,Buyer,Price,...,SEX,Unnamed: 13,Unnamed: 14,Unnamed: 11,AREA_ID,DamSire,Hip #,UT Time,In-Foal-Sire,Under Tack
0,1,Magical Masquerade,M,98,Unbridled,Nannerl,Tiz Wonderful,"Beth Bayer, Agent X",HARTLEY/DE RENZO THOROUGHBREDS,9000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Storm Savvy,M,4,Suave Prospect,National Finalist,Benny the Bull,"Sue Vacek, Agent IV",4500,NOT SOLD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Great Looking Miss,M,93,Great Above,Native Look,D'wildcat,"Kaizen Sales, Agent for Plumley Farms (Continu...",0,NOT SOLD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Heiress,M,7,Wildcat Heir,Note to Mimi,Montbrook,"Beth Bayer, Agent VII",H&E RANCH,10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Bucett,M,94,Buckaroo,Pannie's Sister,Graeme Hall,"Thoroughstock, Agent II",3500,NOT SOLD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
col = df.columns
col

Index(['hip#', 'NAME', 'sex', 'year', 'SIRE', 'DAM', 'IN FOAL SIRE',
       'CONSIGNOR', 'Buyer', 'Price', 'Unnamed: 10', 'OBSDate', 'BUYER',
       'PRICE', 'PS', 'WORK TIME', 'Name', 'Color', 'Sex', 'Year', 'Sire',
       'Dam', 'In Foal Sire', 'Last Bred', 'Sort By Dam', 'Consignor',
       'Area ID', 'Barn', 'Buyer ', 'Price ', 'AlphSort', 'Foal Date',
       'Dam Sire', 'work time', 'set', 'day', 'Status', 'Out Date', 'Status ',
       'out date', 'color', 'mo', 'date', 'yr', 'Unnamed: 12', 'Unnamed: 8',
       'Work Time', 'm', 'd', 'state', 'BARN', 'consignsort', 'Hip#', 'M', 'D',
       'YR', 'State', 'Consignsort', 'BUYER ', 'Unnamed: 17', 'COLOR', 'SEX',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 11', 'AREA_ID', 'DamSire',
       'Hip #', 'UT Time', 'In-Foal-Sire', 'Under Tack'],
      dtype='object')

In [6]:
# hips = ['Hip#','Hip #', 'hip#']
# uts = ['UT Time','Under Tack', 'work time', 'WORK TIME', 'Work Time']
# df['Hip #'] = df['Hip #'].astype(str)
# unnamed = df.columns[df.columns.str.startswith('Unnamed:')]
# print(df[hips].dtypes)
# print(df[uts].dtypes)

Hip#     object
Hip #    object
hip#     object
dtype: object
UT Time       object
Under Tack    object
work time     object
WORK TIME     object
Work Time     object
UT Time       object
dtype: object


In [77]:
df1 = df

cols = ['Price ', 'Price', 'PRICE']
for c in cols:
    df1[c] = df1[c].fillna('')
    df1[c] = df1[c].astype(str)
    df1[c] = df1[c].replace('nan','')
    
df1['Pricef'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

df1['Pricef'] = df1['Pricef'].str.replace(',','')
df1 = df1[df1['Pricef'] != 'Out']
df1 = df1[df1['Pricef'] != 'OUT']
df1 = df1[df1['Pricef'] != 'Not Sold']
df1 = df1[df1['Pricef'] != 'NOT SOLD']

df1['Pricef'] = pd.to_numeric(df1['Pricef'], errors='coerce')

In [78]:
#df1 = df
unwanted = df1.columns[df1.columns.str.startswith('Unnamed:')]
df1.drop(unwanted, axis=1, inplace=True)

cols = ['Hip#', 'hip#', 'Hip #']
#df1['Hipf'] = np.nan
df1['Hipf'] = df1[cols].fillna('').astype(str).replace('nan','').sum(axis=1)
df1 = df1.drop(cols, 1)
df1['Hipf'] = pd.to_numeric(df1['Hipf'], errors='coerce', downcast='unsigned')
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df1.dropna(subset=['Hipf'])

cols = ['DamSire', 'Dam Sire']
df1['DS'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['Buyer', 'BUYER', 'BUYER ']
df1['Buyerf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['Sex', 'sex', 'SEX']
df1['Sexf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['BARN', 'Barn']
df1['Barnf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['Consignor', 'consignsort', 'Consignsort', 'CONSIGNOR']
df1['Consignorf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['Color', 'COLOR', 'color']
df1['Colorf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['Status ', 'Status']
df1['Statusf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['NAME', 'Name']
df1['Namef'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['IN FOAL SIRE', 'In Foal Sire', 'In-Foal-Sire']
df1['IFSf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['WORK TIME', 'work time', 'Work Time', 'Under Tack', 'UT Time']
df1['UTf'] = df1[cols].fillna('').astype(str).sum(axis=1).replace('',np.nan)
df1 = df1.drop(cols, 1)

cols = ['DAM', 'Dam']
df1['Damf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['SIRE', 'Sire']
df1['Siref'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['M', 'm', 'mo']
df1['Mf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['yr', 'YR', 'year', 'Year']
df1['Yf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['out date', 'Out Date']
df1['ODf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

# cols = ['D', 'd', 'day']
# df1['Day'] = df1[cols].fillna('').sum(axis=1)
# df1 = df1.drop(cols, 1)

cols = ['AREA_ID', 'Area ID']
df1['Areaf'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

cols = ['State', 'state']
df1['Statef'] = df1[cols].fillna('').sum(axis=1)
df1 = df1.drop(cols, 1)

col = df1.columns
col

Index(['OBSDate', 'PS', 'Last Bred', 'Sort By Dam', 'Buyer ', 'AlphSort',
       'Foal Date', 'set', 'day', 'date', 'd', 'D', 'Pricef', 'Hipf', 'DS',
       'Buyerf', 'Sexf', 'Barnf', 'Consignorf', 'Colorf', 'Statusf', 'Namef',
       'IFSf', 'UTf', 'Damf', 'Siref', 'Mf', 'Yf', 'ODf', 'Areaf', 'Statef'],
      dtype='object')

In [87]:
df1.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\soldresults10yr707.csv')

In [38]:
dfJune = df1[df1['OBSDate'] == date(year=2021, month=6, day=1)]

In [42]:
dfJune.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\soldresultsJune21.csv')